In [35]:
%reload_ext autoreload
%autoreload 2
import sys
import os
import importlib
script_dir = os.getcwd()  
sys.path.append(os.path.abspath(os.path.join(script_dir, '..')))

In [36]:
import src.config as config

In [37]:
from datetime import datetime, timedelta
import pandas as pd
import pytz

# get current date rounded to hour
current_date = pd.to_datetime(datetime.now(pytz.utc)).floor("H").replace(tzinfo=None)  # DEPCREATED
print(f"{current_date=}")

fetch_data_to = current_date  # date to start fetching rides from
fetch_data_from = current_date - timedelta(days=28)  # date to end fetching rides 



current_date=Timestamp('2024-07-02 11:00:00')


In [51]:
# Simulate call to data warahouse
from src.data import load_raw_data

def fetch_batch_raw_data(from_date, to_date):
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date=}, {to_date_=}')

    # download 2 files from website
    # extract data from from-date's year and months, exclude rows in rides-df whose pickup-datetime-column is after the from-date
    rides = load_raw_data(year=from_date_.year, months=from_date_.month)
    rides = rides[rides.pickup_datetime >= from_date_]
    # extract data from to-date's year and months, exclude rows in rides-df whose pickup-datetime-column is before the to-date
    rides_2 = load_raw_data(year=to_date_.year, months=to_date_.month)
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]

    # combine both dataframes
    rides = pd.concat([rides, rides_2])

    # shift the data to pretend this is recent data
    rides["pickup_datetime"] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides


In [52]:
rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)

from_date=Timestamp('2024-06-04 11:00:00'), to_date_=Timestamp('2023-07-04 11:00:00')
[6]
File 2023-06 is already in local storage
[7]
File 2023-07 is already in local storage


In [56]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

ts_data['pickup_location_id'] = ts_data['pickup_location_id'].astype('int64')  # convert to column to right data type in feature store


100%|██████████| 258/258 [00:00<00:00, 1301.81it/s]


In [57]:
import hopsworks

# connect to the project, using created api/project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/835736
Connected. Call `.close()` to terminate connection gracefully.


In [58]:
feature_group.insert(ts_data, write_options={"wait_for_job": True})

Uploading Dataframe: 0.00% |          | Rows 0/173376 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_hourly_feature_group_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/835736/jobs/named/time_series_hourly_feature_group_3_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x16caed010>, None)